# Split-apply-combine - with SQL and _pandas_
The split-apply-combine processing pattern is also sometimes referred to as a data grouping operation. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

The split-apply-combine processing segments a dataset (split), applies some processing to each segment (apply) and then brings the results back into a single results set (combine).

When handling data this allows different processing to be applied to different subsets of the data, depending on the properties of the subset, but still to treat the data as a single whole collection.

Both SQL and _pandas_ realise the split-apply-combine pattern using  a mechanism based on 'grouping', which segments a dataset by supplying an attribute list whose values determine the membership of each subset.  

We've seen SQL and _pandas_ statements that SELECT rows of data on some condition - say `CourseCode ='TM351'`. SELECTing will extract those rows from the dataset; grouping keeps those rows as a subset within the complete dataset.

In this Notebook we're going to look briefly at the SQL GROUPBY and HAVING clauses and then consider the _pandas_ `groupby()` method over DataFrames.

# Split-apply-combine in SQL

### The GROUP BY clause

The SQL GROUP BY clause takes a list of column names and, using the values in those columns, produces an internal form of the table with data grouped into subgroups.  There is one subgroup for each unique combination of values from those rows.

There is a limitation in SQL that once a GROUP BY has been applied the result returned can have, at most, one row per group.

This requires that any SELECT statement therefore contains combinations of:
- the columns that appeared in the GROUP BY
- aggregate functions over the groups
- constant values.

We'll use _pandasql_ for our SQL examples, but this should extend to other SQL implementations.

In [ ]:
# Start by importing the sqldf function from pandasql:
from pandasql import sqldf

# Then create a simple wrapper function to allow us to supply the query 'q' without 
# the surrounding syntax.
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
# Our sample dataset is in the CSV file 'mixed_module_data' in the data folder
mixed_module_data = pd.read_csv('data/mixed_module_data.csv')
mixed_module_data.head(5)

In [ ]:
# We can apply the GROUP BY to our table, and using the select ModuleCode we can see the values
#  of ModuleCode that segment the data. There's a group for each of these values.
query = '''
SELECT ModuleCode as 'These are the groups in ModuleCode' 
FROM mixed_module_data 
GROUP BY ModuleCode; 
'''
result = pysqldf(query)
result

### GROUP BY and aggregate functions

In [ ]:
# We know what happens if we apply an aggregate function to the whole table.
query = ''' SELECT COUNT(*) as 'how many rows' FROM mixed_module_data; '''
result = pysqldf(query)
result

In [ ]:
# The GROUP BY segments the whole table into groups, 
# and the aggregate function is applied to each group.
query = ''' 
SELECT COUNT(*) as 'how many rows' 
FROM mixed_module_data 
GROUP BY ModuleCode; 
'''
result = pysqldf(query)
result

In [ ]:
# Maybe with some more useful labelling we can see what is happening.
query = ''' 
SELECT ModuleCode, COUNT(*) as 'how many student/marks, grouped by module' 
FROM mixed_module_data 
GROUP BY ModuleCode; '''
result = pysqldf(query)
result

### Exercise
What do you think will happen if we group by `ModuleCode` and `Year`? 
What does this grouping represent?

In [ ]:
# Here's the SQL code to achieve this, 
# but before you run the cell - what do you think will happen?

query = ''' 
SELECT ModuleCode, Year, COUNT(*) as 'how many student/marks' 
FROM mixed_module_data 
GROUP BY ModuleCode, Year; '''
result = pysqldf(query)
result

### Discussion
The groups are now formed from the unique combinations of values in the two named columns.
This probably (remember we don't have the interpretation for the dataset, so we're guessing) represents how many students completed a module in each year.

### Exercise

In [ ]:
# Your turn: write a query, using GROUP BY, that shows how many module marks each student has.
query = ''' 
SELECT <your statement here> 
FROM <and here> 
GROUP BY <and here?>; 
'''
result = pysqldf(query)
result

In [ ]:
# Sample solution
query = ''' 
SELECT Student, COUNT(ModuleCode) 
FROM mixed_module_data 
GROUP BY Student; 
'''
result = pysqldf(query)
result

In [ ]:
# We can apply different aggregate functions to the groups.
query = ''' 
SELECT ModuleCode, COUNT(*), AVG(Mark), Max(Mark), Min(Mark) 
FROM mixed_module_data 
GROUP BY ModuleCode; '''
result = pysqldf(query)
result

In [ ]:
# We can apply different aggregate functions to the groups, however they are formed.
query = ''' 
SELECT ModuleCode, Year, COUNT(*), AVG(Mark), Max(Mark), Min(Mark) 
FROM mixed_module_data 
GROUP BY ModuleCode, Year; 
'''
result = pysqldf(query)
result

### HAVING: conditions on groups
SQL also has a HAVING clause which applies the same selection condition to each group formed by a GROUP BY clause.  It's a bit like the WHERE clause, but is applied to groups not rows.

In [ ]:
# Let's pick out only modules with an average mark higher than 50.
query = ''' 
SELECT ModuleCode 
FROM mixed_module_data 
GROUP BY ModuleCode
HAVING AVG(mark) > 50; 
'''
result = pysqldf(query)
result

In [ ]:
# The HAVING clause applies to the group, so the condition will 
# depend on aggregate functions, or is applied to the grouping columns: 
# it can't refer to individual rows of data, just the groups.
query = ''' 
SELECT ModuleCode 
FROM mixed_module_data 
GROUP BY ModuleCode
HAVING AVG(mark) > 50 and ModuleCode LIKE ('%4'); 
'''
# recall the LIKE condition is a string pattern matching operator - 
# this LIKE condition is true for any module code that ends in a 4.
result = pysqldf(query)
result

### GROUP BY and HAVING in more complex SQL
HAVING only makes sense if there is a GROUP BY clause in the SQL statement - you can't apply a condition to a group if there are no groups.
But GROUP BY can appear with other SELECT clauses - WHERE and ORDER BY, for example.


### Exercise
What do you think the output from the following query represents for this dataset?

In [ ]:
# Complex use of grouping.
query = ''' 
SELECT 'above' as 'banding', ModuleCode, count(student), AVG(mark) 
FROM mixed_module_data 
WHERE mark >= 40 
GROUP BY ModuleCode 
UNION 
SELECT 'below', ModuleCode, count(student), AVG(mark) 
FROM mixed_module_data 
WHERE mark < 40 
GROUP BY ModuleCode 
ORDER BY ModuleCode; 
'''
result = pysqldf(query)
result

### Discussion
The data shows, for each module, the number of students above/below 40 marks and the average mark for those students on that module.

The `banding = 'above'` rows are data about those students who achieved at least 40 marks.

The `banding = 'below'` rows are data about those students who got less than 40 marks.


# Summary: SQL GROUP BY and HAVING
SQL tables can be segmented based on the values in one or more named columns using the GROUP BY clause.

The HAVING clause applies a condition to each group (segment) which, like the WHERE applied to rows, determines if that group appears in the resulting table.

Once grouped, the table cannot be 'ungrouped'; so individual rows are lost and results represent one row per group. This means that the SELECT statement should only contain the grouping column names, aggregate functions, or constant values.

# Split-apply-combine in _pandas_

The _pandas_ library (as we've discovered) offers much richer data manipulation than SQL.  

In _pandas_ we can do what the SQL GROUP BY and HAVING clauses do.  We can also go further than that allowing groups to be formed from more than just the values in the DataFrame.  We can, for example, apply a function to column values to determine groupings and we can choose to group by rows or by columns.  There is also a wide range of ways to manipulate and control the indexing applied to the data collections that are returned from the grouping method.  Unfortunately, in this module, we can only scratch the surface of what is available in the _pandas_ grouped data structures.

In this Notebook we'll look at the grouping using columns from a DataFrame, at the equivalent of the GROUP BY, aggregation and HAVING in SQL, and also look at a feature not easily achieved in SQL, the ability to transform the individual rows of a group based on properties of the group.

There are several online tutorials covering the wider application of the _pandas_ grouping, and the usual _pandas_ documention gives several examples. 

Let's start with looking at what happens when we apply the `groupby()` method to a DataFrame, before looking in more detail at the grouped objects returned.

We still have the `mixed_module_data` DataFrame we read in earlier - so that will be our sample data.


## The `groupby()` method

In [ ]:
# Let's apply groupby over the ModuleCode column and see what we get.
groupeddata = mixed_module_data.groupby(['ModuleCode'])
groupeddata

In [ ]:
# Hmmm, the result is too complex just to display the data 
#            structure - so pandas tells us it is an object.

# What is returned is actually a list of two-valued-tuples, where the 
# first value is the group 'key' value, 
# and the second value is a DataFrame of the rows in that group.

# So we can pull those two values out with a loop construct
for groupkey, grouprows in groupeddata:
    print(groupkey)
    print(grouprows)

In SQL we had lost the ability to see the rows in the group at this point. The object returned by `groupby()` retains those rows until we decide what we want to do with them.  

Often we're going to let the object methods handle the processing of the groups - we can look to see what methods can be applied to the `groupeddata` object that was returned above, using the `<tab>` facility after the `<objectname>`.  In the next cell type `<tab>` after the `.`, then choose the `groups` attribute - this shows us the group keys, and a list of the original index rows that are in that group.

In [ ]:
groupeddata.groups

In [ ]:
# And we can pick out a named group, getting a DataFrame as output.
groupeddata.get_group('QQ225')

We can also supply a list of columns to use as the key for the `groupby()` and this forms a composite grouping key.

In [ ]:
twolevelgroupeddata = mixed_module_data.groupby(['ModuleCode','Year'])
twolevelgroupeddata.groups

# `groupby()` and aggregations
There are two ways to apply aggregate functions to grouped data as the `groupby()` is applied, and,  using the `aggregate()` or `agg()` method on the grouped result.

In [ ]:
mixed_module_data.groupby(['ModuleCode']).count()

In [ ]:
groupeddata.aggregate('count')

The function is only applied to those rows where it makes sense to do so.

In [ ]:
#mixed_module_data.groupby(['ModuleCode']).sum()
groupeddata.aggregate('sum')

In [ ]:
# Or we can pick out the column on which to apply the aggregate 
#   function in the usual pandas way.
groupeddata['Mark'].aggregate('mean')

In [ ]:
# The same for multi-column keys too.
twolevelgroupeddata['Mark'].aggregate('mean')

And the `aggregate()` method gives us an ability to fine-tune the application of aggregate functions to columns.

In [ ]:
# More than one function applied to each column.
groupeddata.aggregate(['max','min','mean'])

Note that because there are multiple results columns involved, the result has a hierarchical column index.   If only a single non-key column is the target then the column label goes away.

In [ ]:
groupeddata['Mark'].agg(['max','min','mean'])

In [ ]:
# We can give names to the resulting columns: using a dict where 
#    the dict key becomes the column name, 
#    the dict value is the function name.
groupeddata['Mark'].agg({'top mark':'max', 
                         'low mark':'min',
                         'average mark':'mean'})

In [ ]:
# And we can apply different functions to different columns.
groupeddata.agg({'Mark':{'top mark':'max'}, 
                 'Year':{'first year':'min'}, 
                 'Student':'max'})
# Each function is applied only to the named column.

We can use `describe()` on the grouped result.

In [ ]:
groupeddata.describe()

In [ ]:
# And plot() generates a plot for each group.
#           BUT you have to be sure to pick out meaningful x and y columns.
# CAREFUL; that's one PLOT per GROUP: that's a lot of plots if you have a lot of groups!

groupeddata.plot.bar(x='Student', y=['Mark'], ylim=(0,100))




# `filter()`: _pandas_' version of HAVING,
## but not quite
In SQL the HAVING clause acted as a filter on the groups: if a group returned True for the condition in the HAVING clause then the group appeared in the result.

_pandas_ has the `filter(<a function applied to a group returning True or False\>)` method which applies the function to the group, **BUT** this returns the original rows from the groups that satisfy the function.  

So what you get back from `filter()` is a new DataFrame with only the rows from the original DataFrame that were in the groups that satisfy the function.

In [ ]:
# We need to define the function, over the specific columns of the group, 
# so it will act as the filter function - i.e. return true or false:
filter_average_mark_over_50 = lambda x: (x['Mark'].mean() > 50)

In [ ]:
groupeddata.filter(filter_average_mark_over_50)

In [ ]:
# It might be easier to see what is happening with a smaller result.
# This function is true if a group has fewer than five members 
#      (x is the group, len(x) is how many elements in the group):
filter_small_modules = lambda x: len(x) < 5

In [ ]:
# So the following reports the rows of any modules that have fewer than five students
smallmodulerows = groupeddata.filter(filter_small_modules)
smallmodulerows

In [ ]:
# If we want the groups that these represent, we need to reapply the groupby after the filter:
groupeddata.filter(filter_average_mark_over_50).groupby(['ModuleCode']).groups

In [ ]:
groupedsmallmodules = smallmodulerows.groupby(['ModuleCode'])
groupedsmallmodules.groups

# `transform()` 
### Altering the rows of a group based on a group condition or group value

This is something SQL can't do without a bit of additional work: affect the rows in a group, based on a property of the group.  

`transform()` _does not change_ the original data.  Instead it generates a DataFrame based on the original, without the grouping columns and with updated column values, with one row for each row in the original and the corresponding rows retaining the same index values as the original dataset.  
We therefore know that rows in the original and the rows in the transformed columns are aligned by index values.

`.transform()` applies the transform function to EVERY column of the DataFrame - so you need to make sure the transform is only applied to meaningful columns of the grouped DataFrame.

### Example
Suppose we don't like modules where the average mark is below some threshold, and we'd like to adjust student marks if the module average is below that mark.

So, how can we ensure the average mark for a group of students on a module is always at least 50?
We start by grouping against the `ModuleCode`, then we find each group's mean. 

If the group mean is below 50 we want to adjust each student mark for students in that group so the group mean is 50.  This gives us the adjusted marks. We then take the adjusted marks result and add that column back to the original data, with a sensible name.   

If the group mean is 50 or above then for all students in that group we want to copy across the student's original mark untouched.

In [ ]:
# First we need a transformation function that 
#    returns 
#         the original individual Mark if the group average is 50 or above, 
#    and returns  
#         (50 - group average) + individual Mark 
#                                        if the group average is below 50
def make_mean_50_if_below(x):
    if x.mean() >= 50:  # These groups are OK.
        return x
    else:               # These groups need to be adjusted.
        return (50-(x.mean())) + x

# When reading the above you need to remember that x will be a Series of individual student marks.

In [ ]:
# We group the original mixed module data, 
#    remembering only to include the group key (ModuleCode), 
#    and the column (Mark) we want to transform.
#    Also remember the index column is carried from the original mixed_module_data DataFrame.
groupedoriginal = mixed_module_data[['ModuleCode','Mark']].groupby(['ModuleCode'])
groupedoriginal.groups

In [ ]:
# Now create a new DataFrame with the updated rows based on the transform function's output. 
rowsupdated = groupedoriginal.transform(make_mean_50_if_below)
rowsupdated.head(4)

In [ ]:
# Now we want the above Series to be added to the original data, showing the adjusted values.
# Make a copy of the original dataset; we could change the original dataset directly, 
#                                      but this way keeps the original data clean.
updateddata = mixed_module_data.copy()

# Add the transformed Mark as a new column,
updateddata['UpdatedMark'] = rowsupdated['Mark']

# and regroup,
groupedupdateddata = updateddata.groupby('ModuleCode')
# and a check that the groups have the same rows (we haven't lost or gained anything).
groupedupdateddata.groups

In [ ]:
# A quick plot of each group showing the mark and updated mark will show 
# that some groups' values are transformed, and others aren't.
groupedupdateddata.plot.bar(x='Student', y=['Mark','UpdatedMark'])

# (hmm, for some reason the first plot is repeated - probably need to report
#  this bug to the developers!)

In [ ]:
# And finally, to confirm the average transformed mark for each group is now 50 or above
# and the original Mark is unchanged
groupedupdateddata['UpdatedMark','Mark'].mean()

### The transform function is applied to _every_ column where it is meaningful, according to the datatypes.

Here's what happens if we set the group average mark quite high (say 3000!) and apply this function to the whole original DataFrame

In [ ]:
def make_mean_3000_if_below(x):
    if x.mean() >= 3000:
        return x
    else:
        return (3000-(x.mean())) + x

In [ ]:
sillygroupedoriginal = mixed_module_data.groupby(['ModuleCode'])

In [ ]:
sillyrowsupdated = sillygroupedoriginal.transform(make_mean_3000_if_below)
sillyrowsupdated.head(10)

As you can see, since `Year` is numeric, and the average of the `Year` in each group is about 2015, then the year gets transformed too!  As the `Student` code is non-numeric the `transform()` is not applied to that column.

# Summary: _pandas_ split_apply_combine
_pandas_ allows DataFrames and Series to be grouped, and the rows within those groups remain accessible.  This allows for a richer range of processing (apply) to build on the result of the split part of split_apply_combine.  

As with SQL, aggregate functions are available to summarise groups but unlike SQL the group filtering (`filter()`) returns the rows in the groups that satisfied the condition.  

Similarly the ability to transform values based on properties of the group as a whole will return ungrouped data, but the index values ensure transformed data can be aligned to the original datasets.

Additional documentation for the _pandas_ `groupby()`, `filter()` and `transform()` can be found at [Group By: split-apply-combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html).


# Exploring some data with grouping, cross tabs and pivot tables
 
You may want to work on one or two of the following to help your understanding of the split_apply_combine processing (and the quick and easy cross tabulation and pivot tables). 

### You probably don't need to do all of the tasks in the list below.

In the `data` folder there is a `salesbook.csv` file.  

It's a fairly boring sales ledger of sales records showing for each date (watch the format!) the location of the sales team member, the sales person's name, their sales team, and what was sold (how many and at what unit price).  Note that each salesperson is in only one team, and no team has two or more people with the same name.

This would be a good time to explore the OpenStudio facilities to share Notebooks for comment by other students.


In [ ]:
!ls data

## Your tasks 

Load the DataFrame from the `CSV` file.

You might want to clean up the data a bit. One suggestion is to split the month/day/year, or create functions to do it. (All sales are in 2015 so we could drop that info and we're not interested in the day of the month of the sale, but we are going to be interested in monthly sales - so we want to retain the month values.)

You might also want to add a sale amount column (units \* item cost) to save recalculating each time it's required, as we will be using this value later.

Given the above data, and the ability to group datasets, write code to:

- a) Show a count of the number of sales records for each District.
- b) Show a count of the number of sales records for each Team in each District, including the Team and District margin totals.
- c) Show the total sales value for each Team in each District summed over the year.
- d) Show the total sales value for each Team Member in each District over the year, showing the District and Team member margin totals. (Remember you need the team name and salesperson name to identify each person uniquely.)
- e) Show a bar chart of the number of sales each month. 
- f) Show a bar chart of the total sales each month.
- g) Show a scatter plot showing the Item Cost v. the number of Units in each record.
- h) Add a Season column to the DataFrame. For each sale record, the value for Season will be derived from the month: (11,12,1) are Winter, (2,3,4) are Spring, (5,6,7) are Summer, (8,9,10) are Autumn. From the sales in each Season calculate the number, average, maximum, minimum and total sale amount over the season (that is, from all the sales records grouped by season report the number of records, and the average, maximum, minimum and total sales amounts).


In [ ]:
# Start by reading in the CSV file.


In [ ]:
# Now add the SaleAmount column.


In [ ]:
# Now add a Month column.


- a) Show a count of the number of sales records for each District.


- b) Show a count of the number of sales records for each Team in each District, including the Team and District margin totals.


- c) Show the total sales value for each District summed over the year.


- d) Show the total sales value for each Team in each District over the year, showing the District and Team margin totals.


- e) Show a bar chart of the number of sales each month.


- f) Show a bar chart of the total sales each month.

- g) Show a scatter plot showing the Item Cost v. the number of Units in each sales record.

- h) Add a Season column to the DataFrame. For each sale record, the value for Season will be derived from the month: (11,12,1) are Winter, (2,3,4) are Spring, (5,6,7) are Summer, (8,9,10) are Autumn. From the sales in each Season calculate the number, average, maximum, minimum and total sale amount over the season (that is, from all the sales records grouped by season report the number of records, and the average, maximum, minimum and total sales amounts).


### Discussion
Possible solutions for these tasks is given in the Notebook `04.5.soln SalesTeamExploration`; remember there may be more than one way to get the above results but the results should be the same.  

When you get a working solution you might want to share it in OpenStudio, and compare your solution with how other students solved the task.

## What next?

If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, move on to `04.6 Introducing regular expressions`.